# Tutorial - Librería pymysql

En este tutorial, aprenderemos lo básico sobre la librería pymysql. Para ello, es necesario conocer lo básico de **Python** y de los comandos disponibles en **MySQL**.

**Importante:** La mayoría de estos ejemplos se hicieron con una base de datos existentes, llamada "sl_marketplace". Se recomienda reemplazarla por una que exista en sus computadores, e ir creando tablas con los comandos vistos en la parte final del tutorial para poder entender a fondo todas las funcionalidades expuestas en este documento.

En primer lugar, importamos la librería (que debe ser instalada previamente).

In [1]:
import pymysql # Librería para trabajar desde Python con bases de datos MySQL

Ahora, nos conectamos con nuestra base de datos. En este caso, asumimos que se tiene una base de datos **local**. Todos los valores, en este caso, son estándar, salvo por "db", que corresponde al nombre de la base de datos utilizada para la realización de este tutorial. Notar que passwd="" ya que en nuestra configuración no tenemos introducida una contraseña.

In [2]:
connection = pymysql.connect(host="localhost", port=3306, user="root", passwd="", db="sl_marketplace")

Ahora, creamos el **cursor**. Este corresponde al iterador de las tablas. A través de él, conseguimos las filas de las tablas que consultamos.

In [3]:
cursor = connection.cursor()

Ahora, hacemos el uso del método **execute**. Este es el que nos permitirá ejecutar nuestras consultas. Haremos uso de una tabla existente en la base de datos de prueba, llamada "stores" para ejemplificar el comando.

In [4]:
cursor.execute("SELECT * FROM stores LIMIT 5;")

5

También se puede pasar la consulta a través de una variable. En este caso, la variable "query".

In [5]:
query = "SELECT * FROM stores LIMIT 5;"
cursor.execute(query)

5

Para revisar las filas de la consulta, hacemos uso del método **fetchall**. Este nos retorna la lista de todas las filas. Notar que, en este caso, las filas corresponden a una tupla, por lo que **no se pueden editar**.

In [6]:
for row in cursor.fetchall():
    print(row)

(1, 'Urbanista', 'Superior/65/109/59', 'alissa.phaeton', 42, 'November 02, 2003')
(2, 'House of RFyre LLC', 'Isle%20RFyre/97/119/23', 'raven.pennyfeather', 473, 'September 10, 2004')
(3, 'sachi Vixen', 'Genesis/100/116/43', 'sachi.vixen', 159, 'May 29, 2005')
(4, "Nyte'N'Day", '', 'nyte.caligari', 831, 'May 06, 2005')
(5, 'Amplify Your Awesome Shop', 'Here/43/52/26', 'torley.linden', 1, 'September 15, 2004')


También podemos iterar haciendo uso de **fetchone**, que básicamente retorna la primera fila de la consulta, y pasa a la siguiente.

In [7]:
cursor.execute(query) # Ejecutamos la consulta de nuevo.
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(1, 'Urbanista', 'Superior/65/109/59', 'alissa.phaeton', 42, 'November 02, 2003')
(2, 'House of RFyre LLC', 'Isle%20RFyre/97/119/23', 'raven.pennyfeather', 473, 'September 10, 2004')
(3, 'sachi Vixen', 'Genesis/100/116/43', 'sachi.vixen', 159, 'May 29, 2005')
(4, "Nyte'N'Day", '', 'nyte.caligari', 831, 'May 06, 2005')
(5, 'Amplify Your Awesome Shop', 'Here/43/52/26', 'torley.linden', 1, 'September 15, 2004')


Por otra parte, está el comando **fetchmany**, que retorna solo la cantidad de filas que deseamos.

In [8]:
cursor.execute(query) # Ejecutamos la consulta de nuevo.
for row in cursor.fetchmany(size = 3):
    print(row)

(1, 'Urbanista', 'Superior/65/109/59', 'alissa.phaeton', 42, 'November 02, 2003')
(2, 'House of RFyre LLC', 'Isle%20RFyre/97/119/23', 'raven.pennyfeather', 473, 'September 10, 2004')
(3, 'sachi Vixen', 'Genesis/100/116/43', 'sachi.vixen', 159, 'May 29, 2005')


Variando la consulta, el tamaño de nuestra fila varía, naturalmente.

In [9]:
cursor.execute("SELECT store_id FROM stores LIMIT 5;") # Seleccionamos solo una columna.
for row in cursor.fetchall():
    print(row)

(1,)
(2,)
(3,)
(4,)
(5,)


También podemos seleccionar valores directamente desde los índices de la tupla resultante.

In [10]:
cursor.execute("SELECT store_id, number_of_items FROM stores LIMIT 5;") # Dos columnas.
for row in cursor.fetchall():
    print(row[1]) # Nos quedamos con la segunda.

42
473
159
831
1


Otro beneficio, es que con pymysql también podemos **crear** y **modificar** tablas. A continuación, un ejemplo práctico.

In [11]:
# Tabla de dos columnas: val1 (numérico) y val2 (caracteres).
# Notar que CHARSET hará que no tengamos problemas con letras del español (como la ñ).
cursor.execute("CREATE TABLE testing (val1 int, val2 varchar(128)) ENGINE=InnoDB DEFAULT CHARSET=latin1;")
# Pasamos los valores en forma de tuplas.
values1 = (1,"Hola, ")
values2 = (2,"mundo!")
cursor.execute("INSERT INTO testing VALUES (%s,%s);",values1)
cursor.execute("INSERT INTO testing VALUES (%s,%s);",values2)

1

Para que las modificaciones sean visualizables en la base de datos, es necesario que **guardemos** los cambios realizados. Por eso, a través de la variable "connection" creada en un comienzo, ejecutamos el commando **commit**.

In [12]:
connection.commit()

Verificamos que todo se haya creado como lo esperábamos, a través de consultas, y un ejemplo práctico que muestra la utilidad que otorga Python para trabajar con bases de datos.

In [13]:
cursor.execute("SELECT * FROM testing;")
result = ""
for row in cursor.fetchall():
    result += row[1]
print(result)

Hola, mundo!


Ahora, modificaremos las tuplas y revisaremos el resultado.

In [14]:
# Otra forma de insertar valores en strings (ojo con el detalle de las comillas simples para introducir el valor de los
# caracteres).
cursor.execute("UPDATE testing SET val2 = '{}' WHERE val1 = 1;".format("Hello, "))
cursor.execute("UPDATE testing SET val2 = '{}' WHERE val1 = 2;".format("world!"))
connection.commit() # Guardamos los cambios.
# Vemos el resultado.
cursor.execute("SELECT * FROM testing;")
result = ""
for row in cursor.fetchall():
    result += row[1]
print(result)

Hello, world!


Finalizaremos este tutorial con la eliminación de la tabla creada recientemente.

In [15]:
cursor.execute("DROP TABLE testing;")
connection.commit()

# Revisamos que, efectivamente, se eliminó.
try:
    cursor.execute("SELECT * FROM testing;")
except:
    # Error - la tabla ya no existe.
    print("¡Tabla eliminada correctamente!")

¡Tabla eliminada correctamente!
